In [5]:
"""
The MIT License (MIT)

Copyright © 2023 Dr Keith S Reid Cailleach Computing Ltd

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and 
associated documentation files (the “Software”), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial
portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

=#

#=

Python version
Python 3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0] :: Anaconda, Inc. on linux

Date 18 Feb 2023

Box Spec

            .-/+oossssoo+/-.               
        `:+ssssssssssssssssss+:`           ------------ 
      -+ssssssssssssssssssyyssss+-         OS: Ubuntu 22.04.1 LTS x86_64 
    .ossssssssssssssssssdMMMNysssso.       Host: 
   /ssssssssssshdmmNNmmyNMMMMhssssss/      Kernel: 5.15.0-60-generic 
  +ssssssssshmydMMMMMMMNddddyssssssss+     Uptime: 
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/    Packages: 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Shell: bash 5.1.16 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   Resolution: 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   DE: GNOME 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   WM: Mutter 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   WM Theme: Adwaita 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Theme: 
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/    Icons: 
  +sssssssssdmydMMMMMMMMddddyssssssss+     Terminal: 
   /ssssssssssshdmNNNNmyNMMMMhssssss/      CPU: AMD Ryzen 9 3900X (24) @ 3.800G 
    .ossssssssssssssssssdMMMNysssso.       GPU: 
      -+sssssssssssssssssyyyssss+-         Memory: 64Gb RAM
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.                                       
                                                                   

=#

#=
Intent:
Implement Stern Brocot in Python and Julia using TDD and nothing too clever
For speed comparison
Roughly simlar logic in the two versions
=#

"""

# 1 Libraries

# base

# 2 Classes

class RationalNode:
    def __init__(self, numerator, denominator, place, layer):
        self.numerator      = numerator
        self.denominator    = denominator
        self.place          = place
        self.layer          = layer

# 3 Config

def get_depth():
    return 3

# 4 model

def count_nodes(depth):
    node_count = (2**depth)-1 # Mersene, n'est pas? 
    return node_count

# 5 view

def show_node(this_node):
    print(this_node.numerator, "\t", this_node.denominator, "\t", this_node.place, "\t", this_node.layer)
    

# 6 control

if __name__=="__main__":
    
    # tree has configurable depth see config
    depth           = get_depth()
    node_count      = count_nodes(depth)
    powers          = []
    over_power      = 2**depth
    this_power      = int(over_power/2)

    while this_power >= 1:
        powers.append(this_power)
        this_power = int(this_power/2)
        
    for current_layer in range(depth):
        this_power      = powers[current_layer]
        if current_layer == 0:
            
            # these predate root and are loop invariants
            zero_over_one   = RationalNode(0,1,0,           current_layer)
            one_over_zero   = RationalNode(1,0,node_count+1,current_layer)
            triangular_part = [RationalNode(None,None,i+1,None) for i in range(node_count)]    
            tree            = [zero_over_one]
            tree.extend(triangular_part)
            tree.extend([one_over_zero])
            
            left_parent     = zero_over_one
            right_parent    = one_over_zero
            
            child               = RationalNode(None,None,this_power,left_parent.layer+1)
            child_numerator     = left_parent.numerator + right_parent.numerator
            child.numerator     = child_numerator
            child_denominator   = left_parent.denominator + right_parent.denominator
            child.denominator   = child_denominator
            tree[child.place]   = child
            child_places        = [child.place]
            
        else:
            grand_child_places  = []
            for parent_place in child_places:
                left_child_place    =  parent_place - this_power 
                right_child_place   =  parent_place + this_power
                grand_child_places.extend([left_child_place, right_child_place])
            child_places = grand_child_places
                
            
            
        
    for i in tree:
            show_node(i)

0 	 1 	 0 	 0
None 	 None 	 1 	 None
None 	 None 	 2 	 None
None 	 None 	 3 	 None
1 	 1 	 4 	 1
None 	 None 	 5 	 None
None 	 None 	 6 	 None
None 	 None 	 7 	 None
1 	 0 	 8 	 0


In [3]:
def test_count_nodes():
    depth = 0
    node_count = count_nodes(depth)    
    assert (node_count == 0), 'inaccurate node count'
    
    depth = 1
    node_count = count_nodes(depth)
    assert (node_count == 1), 'inaccurate node count'
    
    depth = 2
    node_count = count_nodes(depth)
    assert (node_count == 3), 'inaccurate node count'
    
    depth = 10
    node_count = count_nodes(depth)
    assert (node_count == 1023), 'inaccurate node count'
    
    depth = 21
    node_count = count_nodes(depth)
    assert (node_count == 2097151), 'inaccurate node count'
    
def PythonSternBrocotTests():
    test_count_nodes()
    print("passed all tests")
    
PythonSternBrocotTests()

passed all tests
